Usage of the package
========

First, some imports

In [1]:
import rustworkx as rx

from territories import TerritorialUnit, Territory, Partition

## Creation of the tree

The first step is to create a tree of known entities. This can be a very compute intensive task, depending on the tree size.

Here we'll create a very simple tree.

In [2]:

lyon = TerritorialUnit("Lyon")
marseille = TerritorialUnit("Marseille", es_code="COM:2909") # you can specify an ElasticSearch code
paris = TerritorialUnit("Paris")
nogent = TerritorialUnit("Nogent")
pantin = TerritorialUnit("Pantin")
villeurbane = TerritorialUnit("Villeurbane")
sté = TerritorialUnit("Saint Etienne")

metropole = TerritorialUnit("Grand Lyon", False, Partition.DEP)

sud = TerritorialUnit("Sud", False, Partition.REGION)
idf = TerritorialUnit("Île-de-France", False, Partition.REGION)
rhone = TerritorialUnit("Rhône", False, Partition.DEP)

france = TerritorialUnit("France", False, Partition.COUNTRY)


# buiding the reference Part tree
entities = (france, sud, idf, rhone, metropole, nogent, pantin, paris, marseille, sté, villeurbane, lyon)

tree = rx.PyDiGraph()
entities_indices = tree.add_nodes_from(entities)

mapper = {o : idx for o, idx in zip(entities, entities_indices)}
edges = [
    (france, idf),
    (france, sud),
    
    (idf, nogent),
    (idf, pantin),
    (idf, paris),

    (sud, marseille),
    (sud, rhone),

    (rhone, metropole),
    (rhone, sté),

    (metropole, villeurbane),
    (metropole, lyon),
    ]

tree.add_edges_from([
    (mapper[parent], mapper[child], None) for parent, child in edges
])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

Once the tree is created, you need to assign it.

In [3]:
Territory.assign_tree(tree)

Then, you can start to create territories from arbitrary entities.

In [4]:
a = Territory(sté, marseille)
b = Territory(lyon, france)
c = Territory(paris, nogent, pantin, lyon, lyon, metropole)
d = Territory(lyon, villeurbane, marseille)
e = Territory(rhone, idf)
f = Territory(idf, marseille, metropole)

Entities associated territories are represented in an efficient way : if all leaves of a parent node are included in the territory, they are simply replaced by their parent node.

In [5]:
b # since Lyon is included in France, only France remains

France

## Operations on entities

Usual operation on territories works as expected :

In [6]:
# addition

print(a, c)
print(a + c) # This simplify to France

Marseille|Saint Etienne Grand Lyon|Île-de-France
France


In [7]:
# substraction

print(a, d)
print(a - d) # only Saint Etienne remains

Marseille|Saint Etienne Marseille|Grand Lyon
Saint Etienne


More importantly, sets operations are also supported

In [8]:
# intersection
print(f"Intersection of {a} and {d} is {a & d}")

# union
print(f"Union of {c} and {f} is {f | c}")

Intersection of Marseille|Saint Etienne and Marseille|Grand Lyon is Marseille
Union of Grand Lyon|Île-de-France and Marseille|Grand Lyon|Île-de-France is Marseille|Grand Lyon|Île-de-France
